In [1]:
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# считываем данные из файла и преобразуем их в двумерный массив
def read_data(filename):
    import sys
    with open(f'{filename}.txt', 'r') as sys.stdin:
        inp_data = np.array(np.matrix('; '.join(list(map(str.strip, sys.stdin.readlines())))))
        return inp_data

In [3]:
# нормализация
def norming(data):
    for n in range(len(data[0])):
        max_n, min_n = max(data[:, n]), min(data[:, n])
        k, b = 1 / (max_n - min_n), -min_n / (max_n - min_n)
        for m in range(len(data)):
            data[m, n] *= k
            data[m, n] += b

    return data

In [4]:
def p(x, sigma, mu):
    det = np.prod(sigma)
    
    res = 0
    for n in range(len(x)):
        res += (x[n] - mu[n]) ** 2 / sigma[n]
    res /= -2
    
    return np.exp(res) / (np.sqrt(det) * (2 * np.pi) ** (len(x) / 2.0) )

In [5]:
def EM_learn(K, data, tmax):
        
    M = len(data)
    N = len(data[0])
    
    # инициализация
    w_k = []
    for _ in range(K):
        w_k.append(1 / K)
        
    mu_k = []
    for k in range(K):
        mu_k.append(np.copy(data[k]))
        
    sigma_k = []
    for k in range(K):
        sigma = [0] * N
        for n in range(N):
            sigma[n] = (1.0 / (M * K)) * sum([ (data[m][n]  - mu_k[k][n]) ** 2 for m in range(M) ])
            # sigma[n] = 1.0
        sigma_k.append(sigma)
        
    
    gmk = np.zeros((M, K)) 
    for t in range(tmax):
        
        # E-шаг
        for m in range(M):
            wpk = [ w_k[k] * p(data[m], sigma_k[k], mu_k[k]) for k in range(K) ]
            denominator = sum(wpk)
            for k in range(K):
                gmk[m][k] = wpk[k] / denominator
        
        
        # M-шаг
        if tmax - t - 1 > 1:
            
            for k in range(K):
                w_k[k] = (1.0 / M) * np.sum(gmk[:, k])
            
            for k in range(K):
                for n in range(N):
                    mu_k[k][n] = (1.0 / (M * w_k[k])) * sum( [ data[m][n] * gmk[m][k] for m in range(M)] )
                    
            for k in range(K):
                for n in range(N):
                    sigma_k[k][n] = (1.0 / (M * w_k[k])) * sum( [ ((data[m][n] - mu_k[k][n]) ** 2) * gmk[m][k] for m in range(M)] )
    
    return gmk

In [6]:
data = read_data('firms')
print(data)

[[ 1.6867e+01  2.4110e+00  2.0930e+00  2.7047e+01  4.6800e+05]
 [ 1.9100e+00  2.7900e-01  4.5200e-01  2.4600e+00  1.6821e+04]
 [ 4.5200e+00  9.7900e-01  7.7600e-01  6.8600e+00  1.0100e+05]
 [ 5.6400e+00  2.3300e+00  1.2500e+00  4.1200e+01  2.8560e+05]
 [ 2.5600e-01  3.6900e-01 -1.4700e-02  5.1500e-01  1.1846e+04]
 [ 1.4600e-02  1.1800e-01  6.4300e-02  1.0200e+00  5.8415e+04]
 [ 7.7000e-04  1.2000e-03 -9.7000e-05  8.1100e-03  1.2000e+04]
 [ 5.4900e-02  1.0100e-02  8.6000e-03  6.5800e-02  8.3000e+03]
 [ 4.9900e-02  3.5200e-02  7.1800e-03  6.9500e-01  4.5000e+04]
 [ 3.8100e-03  2.2100e-03  1.2320e-03  1.1500e-01  7.7910e+03]]


In [7]:
data = norming(data)
print(data)

[[1.00000000e+00 1.00000000e+00 1.00000000e+00 6.56412949e-01
  1.00000000e+00]
 [1.13198385e-01 1.15279276e-01 2.21426199e-01 5.95236101e-02
  1.96215198e-02]
 [2.67945474e-01 4.05759814e-01 3.75148266e-01 1.66340753e-01
  2.02536239e-01]
 [3.34350356e-01 9.66387252e-01 6.00037956e-01 1.00000000e+00
  6.03658338e-01]
 [1.51326052e-02 1.52626774e-01 0.00000000e+00 1.23055776e-02
  8.81121404e-03]
 [8.19981703e-04 4.84687526e-02 3.74816150e-02 2.45652724e-02
  1.10002195e-01]
 [0.00000000e+00 0.00000000e+00 6.92840537e-03 0.00000000e+00
  9.14584461e-03]
 [3.20937163e-03 3.69325255e-03 1.10547042e-02 1.40051840e-03
  1.10601922e-03]
 [2.91292126e-03 1.41090547e-02 1.03809840e-02 1.66753698e-02
  8.08523953e-02]
 [1.80241820e-04 4.19121919e-04 7.55895051e-03 2.59492827e-03
  0.00000000e+00]]


In [10]:
# К = 2
mat = EM_learn(2, data, tmax = 3)
print(mat)

[[1.00000000e+000 3.75435458e-116]
 [1.75115700e-213 1.00000000e+000]
 [6.46016758e-092 1.00000000e+000]
 [1.00000000e+000 1.47587221e-100]
 [3.72086771e-199 1.00000000e+000]
 [3.46961485e-247 1.00000000e+000]
 [2.81801651e-273 1.00000000e+000]
 [2.43302792e-271 1.00000000e+000]
 [3.00172032e-265 1.00000000e+000]
 [4.50664726e-273 1.00000000e+000]]


In [12]:
# K = 3
mat = EM_learn(3, data, tmax = 3)
print(mat)

[[1.00000000e+000 0.00000000e+000 1.92359544e-069]
 [0.00000000e+000 9.74916613e-001 2.50833871e-002]
 [6.06530934e-245 2.61218619e-022 1.00000000e+000]
 [1.00000000e+000 0.00000000e+000 1.33948829e-059]
 [0.00000000e+000 9.99998799e-001 1.20052914e-006]
 [0.00000000e+000 9.99994008e-001 5.99234702e-006]
 [0.00000000e+000 9.99999940e-001 6.01285408e-008]
 [0.00000000e+000 9.99999938e-001 6.19420685e-008]
 [0.00000000e+000 9.99999335e-001 6.64608055e-007]
 [0.00000000e+000 9.99999944e-001 5.56447473e-008]]
